> **訳注**：このnotebookは，Udacityの[Deep learning nanodegreeの公開教材](https://github.com/udacity/deep-learning)を和訳するプロジェクトの一環として，[Sentiment analysis with TFLearn](https://github.com/haltaro/deep-learning-in-japanese/blob/master/intro-to-tflearn/TFLearn_Sentiment_Analysis_Solution.ipynb)を翻訳したものです（**Exercise**は翻訳対象外です）．本プロジェクトのnotebook一覧は[こちら](https://haltaro.github.io/deep-learning-in-japanese/)．

# TFLearnで評判分析

このnotebookでは，Andrew Traskのプロジェクトに引き続き，映画レビューの評判分析のためのニューラルネットワークを構築します．ただし，Numpyではなく，TensorFlowのラッパーである[TFLearn](http://tflearn.org/)を使います．layerを定義するだけで良いので，TFLearnでネットワークを構築するのは簡単です．TFLearnは，あなたの代わりに細部を実装してくれます．

> **訳注**: Udacityでは，このnotebookの前にAndrew Trask氏のビデオ講座があり，Numpyを使ったニューラルネットワークの実装方法が紹介されました．以降何度か言及があります（「もう一度Andrewのビデオを見てみましょう」など）が，最低限必要な情報は本notebookに含まれていますので，無視しても問題ありません．

さあ，まずは必要なモジュールをimportし，データをロードしましょう．

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical

## データの準備

Andrewの言うとおり，ここでの目標は全てのレビューをword vectorに変換することです．word vectorの各要素は，vocabulary中のwordと対応しています．もし二番目の要素が`the`に対応するならば，全レビュー中の`the`の出現回数をそこに格納します．以下では，実例を用いて説明します．詳細は，Andrewの教材を再度確認してください．

### データの読み込み

pandasを使って，csvファイルからレビューとラベル（ポジティブ，あるいはネガティブ）を読み込みます．このデータは前処理済みであり，全て小文字で記載されています．もし生データから作業を始める場合は，全てのレビューが小文字とは限りませんので，この前に小文字への変換処理が必要です．これは，例えば`The`，`the`，`THE`などを，全て同じ単語として取り扱うためです．

In [2]:
reviews = pd.read_csv('reviews.txt', header=None)
labels = pd.read_csv('labels.txt', header=None)

### 出現頻度の数え上げ

まず，データ中で各単語が何回出現したか数え上げる必要があります．この出現頻度をもとに，テキストデータを数値に変換するためのvocabularyを構築します．この出現頻度集合は，[bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model)と呼ばれます．ここでもう一度Andrewの講義を復習しましょう．以下では，[Counter class](https://docs.python.org/2/library/collections.html#collections.Counter)を使って実装します．

In [3]:
from collections import Counter
total_counts = Counter()
for _, row in reviews.iterrows():
    total_counts.update(row[0].split(' '))
print("Total words in data set: ", len(total_counts))

Total words in data set:  74074


出現頻度の高い10000単語は残しましょう．Andrewの言うとおり，ほとんどの単語は稀にしか出現しないため，予測に大きな影響を与えません．以下では，`vocab`を単語の出現頻度で並び替え，出現頻度の高い10000単語のみ残します．

In [4]:
vocab = sorted(total_counts, key=total_counts.get, reverse=True)[:10000]
print(vocab[:60])

['', 'the', '.', 'and', 'a', 'of', 'to', 'is', 'br', 'it', 'in', 'i', 'this', 'that', 's', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'you', 'on', 't', 'not', 'he', 'are', 'his', 'have', 'be', 'one', 'all', 'at', 'they', 'by', 'an', 'who', 'so', 'from', 'like', 'there', 'her', 'or', 'just', 'about', 'out', 'if', 'has', 'what', 'some', 'good', 'can', 'more', 'she', 'when', 'very', 'up', 'time', 'no']


vocabularyの最後の単語は何ですか？この結果から，vocabularyの単語数の妥当性を判断できます．もし最後の単語の出現頻度が大きい場合は，単語数をより大きくする必要があります．

In [5]:
print(vocab[-1], ': ', total_counts[vocab[-1]])

float :  30


vocabularyの最後の単語は，25000中30のレビューで登場します．これは，全レビューに対して非常に小さな割合と言えます．おそらく，vocaburalyの単語数は適正でしょう．

**Note:**：実行時，上記と異なる単語が出力されるかもしれませんが，頻度は`30`のはずです．これは，`Counter`クラスが，出現頻度が同じ単語の中からどれを返すかは，場合により異なるためです．

データ中の各レビューについて，word vectorを構築します．まず，単語とindexを対応付ける必要がありますが，これは辞書型で簡単に実装できます．

In [6]:
word2idx = {word: i for i, word in enumerate(vocab)}

### Text to vector function

ここでは，文章をword vectorに変換する関数を実装します．この関数は，文字列の入力に対し，各単語の出現回数を記録したvectorを出力します．以下に，そのアルゴリズムを示します．

* word vectorを[np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html)で初期化．ここで，word vectorはvocaburaryと同じ長さ．
* 入力された文字列を`.split(' ')`で単語リストに分割．なお，誤って`.split()`を使うと，意図しない出力となる可能性があるので注意．
* リスト中の全ての単語について，`word2idx`から得られるインデックスの要素をインクリメント．

**Note:** 全ての単語が辞書`vocab`に登録されているわけではないため，いくつかの単語に対してkey errorを返すことがあります．辞書`word2idx`の`.get()`メソッドを使うことで，key error時のデフォルトの返り値を指定できます．例えば，`word2idx.get(word, None)`は，`word`が辞書中に存在しないとき，`None`を返します．

In [7]:
def text_to_vector(text):
    word_vector = np.zeros(len(vocab), dtype=np.int_)
    for word in text.split(' '):
        idx = word2idx.get(word, None)
        if idx is None:
            continue
        else:
            word_vector[idx] += 1
    return np.array(word_vector)

正しく動作すれば，次のように出力されるはずです．

```
text_to_vector('The tea is for a party to celebrate '
               'the movie so she has no time for a cake')[:65]
                   
array([0, 1, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])
```       

In [8]:
text_to_vector('The tea is for a party to celebrate '
               'the movie so she has no time for a cake')[:65]

array([0, 1, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])

さあ，全てのレビューをそれぞれword vectorに変換してみましょう．

In [9]:
word_vectors = np.zeros((len(reviews), len(vocab)), dtype=np.int_)
for ii, (_, text) in enumerate(reviews.iterrows()):
    word_vectors[ii] = text_to_vector(text[0])

In [10]:
# Printing out the first 5 word vectors
word_vectors[:5, :23]

array([[ 18,   9,  27,   1,   4,   4,   6,   4,   0,   2,   2,   5,   0,
          4,   1,   0,   2,   0,   0,   0,   0,   0,   0],
       [  5,   4,   8,   1,   7,   3,   1,   2,   0,   4,   0,   0,   0,
          1,   2,   0,   0,   1,   3,   0,   0,   0,   1],
       [ 78,  24,  12,   4,  17,   5,  20,   2,   8,   8,   2,   1,   1,
          2,   8,   0,   5,   5,   4,   0,   2,   1,   4],
       [167,  53,  23,   0,  22,  23,  13,  14,   8,  10,   8,  12,   9,
          4,  11,   2,  11,   5,  11,   0,   5,   3,   0],
       [ 19,  10,  11,   4,   6,   2,   2,   5,   0,   1,   2,   3,   1,
          0,   0,   0,   3,   1,   0,   1,   0,   0,   0]])

### 訓練，検証，テスト用データセット

`word_vectors`を準備出来たので，次はデータを訓練用，検証用，そしてテスト用に分割します．訓練用データセットはネットワークを訓練するためのもので，検証用データセットはハイパーパラメータを調整するためのもので，テスト用データセットでネットワークの性能を評価するためのものです．TFLearnの`to_categorical`関数を使い，1次元のラベルデータを2次元のベクトルに変換することで，softmax関数で分類できるようにします．後ほどTFLearnがそれに代わる処理を行ってくれるため，ここでは，検証用データセットを作成しません．

In [11]:
Y = (labels=='positive').astype(np.int_)
records = len(labels)

shuffle = np.arange(records)
np.random.shuffle(shuffle)
test_fraction = 0.9

train_split, test_split = shuffle[:int(records*test_fraction)], shuffle[int(records*test_fraction):]
trainX, trainY = word_vectors[train_split,:], to_categorical(Y.values[train_split], 2)
testX, testY = word_vectors[test_split,:], to_categorical(Y.values[test_split], 2)

In [12]:
trainY

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.]])

## ネットワークの構築

[TFLearn](http://tflearn.org/)では，[layers（層）を定義する](http://tflearn.org/layers/core/)ことでネットワークの構築が可能です．

### 入力層

ユニット数を指定するだけで，入力層を作成することができます．例えば，

```
net = tflearn.input_data([None, 100])
```

上記は入力ユニットが100の入力層を作成します．リストの最初の要素である`None`は，バッチサイズを表します．`None`により，デフォルトのバッチサイズを指定できます．

入力ユニット数は，入力するデータサイズと整合する必要があります．例えば，ここでは要素数10000のword vectorを入力しますので，10000の入力ユニットが必要です．

### 層の追加

下記のように，隠れ層を追加できます．

```
net = tflearn.fully_connected(net, n_units, activation='ReLU')
```

上記は，直前の層と全結合する層を追加します．第一引数の`net`は，先ほど`tflearn.input_data`で作成したネットワークです．ここで指定されたネットワークの出力を，新しい層の入力として利用します．`n_units`で新しい層のユニット数を，`activation`で活性化関数を指定できます．`net = tflearn.fully_connected(net, n_units)`を繰り返すことで，全結合層をいくつでも追加できます．

### 出力層

最後に，出力層を追加します．ここでは，ラベルデータと整合するよう，ユニット数を指定する必要があります．ここでは，ポジティブとネガティブの2クラス分類を行いますので，ユニット数は2です．また，問題に適した活性化関数を選択する必要があります．ここでは，ある入力データが2つのクラスのどちらに属するか予測したいので，softmax関数を使います．

```
net = tflearn.fully_connected(net, 2, activation='softmax')
```

### 訓練

下記のように，ネットワークの訓練について指定できます．

```
net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')
```

上記は，構築したネットワーク`net`を引数にとります．また：

* `optimizer`は訓練手法です．ここでは確率的勾配効果法（stochastic gradient descent）を使います．
* `learning_rate`は，学習率です．
* `loss`は，損失関数です．ここでは交差エントロピー（categorical cross entropy）を使います．

最後に，`tflearn.DNN(net)`を使って，一つのモデルとして組み上げます．最終的なコードは，下記のようになります．

```
net = tflearn.input_data([None, 10])                          # Input
net = tflearn.fully_connected(net, 5, activation='ReLU')      # Hidden
net = tflearn.fully_connected(net, 2, activation='softmax')   # Output
net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')
model = tflearn.DNN(net)
```

In [13]:
# Network building
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    # Inputs
    net = tflearn.input_data([None, 10000])

    # Hidden layer(s)
    net = tflearn.fully_connected(net, 200, activation='ReLU')
    net = tflearn.fully_connected(net, 25, activation='ReLU')

    # Output layer
    net = tflearn.fully_connected(net, 2, activation='softmax')
    net = tflearn.regression(net, optimizer='sgd', 
                             learning_rate=0.1, 
                             loss='categorical_crossentropy')
    
    model = tflearn.DNN(net)
    return model

## 初期化

次に，`build_model()`で実際にモデルを構築します．ここでは特に引数を与えませんが，必要に応じてモデルのパラメータを指定することができます．

> **Note:** TFLearnでは，TensorFlowで廃止予定のコードを使用しているため，WARNINGが出る可能性があります．将来的なアップデートで，この問題は解決されるでしょう．たぶん．

In [14]:
model = build_model()

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.merge.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


## 訓練

それでは，構築したネットワーク（`model`）を訓練しましょう．ここでは，`model.fit`メソッドに，訓練用の入力データ`trainX`と訓練用のラベルデータ`trainY`を渡します．下記の`validation_set=0.1`は，訓練用データセットのうち10%を検証用に使うことを意味します．`batch_size`と`n_epoch`で，それぞれバッチサイズとエポック数を指定できます．下記は，ネットワークをword vectorで訓練するためのコードです．

検証精度（validation accuracy）を向上させるために，ハイパーパラメータを変えて`model.fit`を再実行しても良いです．ただし，ハイパーパラメータの調整は，必ず検証用データセットを使うことを覚えておいてください．**テスト用データセットは，訓練終了後の性能評価にのみ使用します．**

In [15]:
# Training
model.fit(trainX, trainY, validation_set=0.1, show_metric=True, batch_size=128, n_epoch=100)

Training Step: 15900  | total loss: 0.28905
| SGD | epoch: 100 | loss: 0.28905 - acc: 0.8768 | val_loss: 0.45618 - val_acc: 0.8351 -- iter: 20250/20250
Training Step: 15900  | total loss: 0.28905
| SGD | epoch: 100 | loss: 0.28905 - acc: 0.8768 | val_loss: 0.45618 - val_acc: 0.8351 -- iter: 20250/20250
--


## 評価

ハイパーパラメータの調整が終了したら，テスト用データセットで性能を評価します．繰り返しになりますが，**テスト用データセットを使えるのは，ハイパーパラメータを決定した後だけです．**

In [16]:
predictions = (np.array(model.predict(testX))[:,0] >= 0.5).astype(np.int_)
test_accuracy = np.mean(predictions == testY[:,0], axis=0)
print("Test accuracy: ", test_accuracy)

Test accuracy:  0.8504


## 遊んでみよう！

In [17]:
# Helper function that uses your model to predict sentiment
def test_sentence(sentence):
    positive_prob = model.predict([text_to_vector(sentence.lower())])[0][1]
    print('Sentence: {}'.format(sentence))
    print('P(positive) = {:.3f} :'.format(positive_prob), 
          'Positive' if positive_prob > 0.5 else 'Negative')

In [18]:
sentence = "Moonlight is by far the best movie of 2016."
test_sentence(sentence)

sentence = "It's amazing anyone could be talented enough to make something this spectacularly awful"
test_sentence(sentence)

Sentence: Moonlight is by far the best movie of 2016.
P(positive) = 0.932 : Positive
Sentence: It's amazing anyone could be talented enough to make something this spectacularly awful
P(positive) = 0.002 : Negative
